# Placement Tokens Demo

This notebook demonstrates placing a job to an AP, using a Placement Token for authorization.

Click on the following link to visit the site where you can obtain a Placement Token: <https://demo-ap.chtc.wisc.edu>

You will be asked to log in with your NetID credentials, then select the permissions for your token.
The commands in this demonstration will require both "read" and "write" permission.
Download the token to a file on your computer.

Run the following cell to create buttons for uploading the token.

In [ ]:
import setup_notebook; setup_notebook.setup()


We will use the `htcondor2` Python library to run HTCondor commands.

In [ ]:
import htcondor2


Run the following cell to create the object named `schedd` with which you will interact with the Access Point.

In [ ]:
collector = htcondor2.Collector()
schedd_ad = collector.locate(htcondor2.DaemonType.Schedd, htcondor2.param["SCHEDD_HOST"])
schedd = htcondor2.Schedd(schedd_ad)


Verify that the connection to the Access Point is functional by querying for jobs.
If the following command does not show any errors, the connection is functional.
(It is normal for the result to be empty if there are no jobs placed at the AP.)

In [ ]:
schedd.query(projection=["ClusterId", "ProcId"])


## Placing a job

Create the `submit` object, which will hold the submit description of a test job that you will place.

In [ ]:
submit = htcondor2.Submit(
"""
Output = test_job.$(Cluster).out
Error = test_job.$(Cluster).err
Log = test_job.$(Cluster).log

Executable = /bin/bash
Arguments = "-c 'sleep 30; echo Job succeeded at `date`'"
Transfer_Executable = false

Queue
"""
)


Place the job.

In [ ]:
placement = schedd.submit(submit, spool=True)
schedd.spool(placement)


Check the status of the job

In [ ]:
query = schedd.query(f"ClusterId == {placement.cluster()}")
